# Webscraping OkadaBooks

N/B: If you are already familiar with webscraping, you can skip to Part 4 that has a code block of the full scraping algorithm.

## Part 1: Getting specific data 

There are 22 categories on okadabooks.com. My aim is to extract all relevant information (Title, Author, No of reads, Rating, Price, Blurb and Link to Book) from each bookcard. Before looping all through the categories, the code for getting each specific content is generated from one of the categories (In this case: fiction).

### Reading the webpage into python

In [1]:
import requests

In [2]:
### Calling headers to imitate browser window
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}

In [3]:
response = requests.get('https://okadabooks.com/category/fiction/5', headers=headers)

In [4]:
### Print first 500 characters of Html
print(response.text[0:500])

<!DOCTYPE html><html><head><title class="next-head">Mystery | OkadaBooks</title><meta charSet="utf-8" class="next-head"/><meta http-equiv="X-UA-Compatible" content="IE=edge" class="next-head"/><meta name="viewport" content="width=device-width, initial-scale=1" class="next-head"/><meta name="description" content="OkadaBooks is a fast, simple and fun way to read books without ever leaving your couch" class="next-head"/><meta name="url" content="https://okadabooks.com/" class="next-head"/><meta nam


### Parsing the html through BeautifulSoup

In [5]:
from bs4 import BeautifulSoup

In [6]:
soup= BeautifulSoup(response.text, 'html.parser')

In [7]:
## On 'https://okadabooks.com/category/fiction/5' , Right-click on a bookcard and click on inspect,
## the take note the class section which highlights the whole bookcard.

bookcards= soup.find_all('div', {'class':'book__card'})

In [8]:
len(bookcards)

12

In [9]:
### Concentrating on the first bookcard to scrap specific data.
bookcard= bookcards[0]

In [10]:
### This helps to arrange the html contect in a structured manner so that the different tags can 
### quickly be gotten.

print(bookcard.prettify())

<div class="book__card">
 <div class="book">
  <div class="book__cover--container">
   <a href="/book/about/magajin_wilbafos_chapter_272_zaren_laid_n/30045">
    <img alt="Magajin Wilbafos [Chapter 272] : Zaren Laidán" src="https://okada-assets-production.s3.eu-west-2.amazonaws.com/applications/content/images/bookImages/73e5b10c1a1b5396348bcd6b96bfc95d.jpg"/>
   </a>
  </div>
  <div class="book__content">
   <h3 class="title">
    <a href="/book/about/magajin_wilbafos_chapter_272_zaren_laid_n/30045">
     <span class="text-truncate">
      magajin wilbafos [chapter 272] : zaren laidán
     </span>
    </a>
   </h3>
   <div class="stats__container">
    <div class="reads">
     <span class="icon">
      <i class="icon ion-md-eye">
      </i>
     </span>
     <span class="text">
      22
      <!-- -->
      <!-- -->
      reads
     </span>
    </div>
    <div class="ratings" title="ratings">
     <span class="icon">
      <i class="icon ion-md-star">
      </i>
     </span>
     <span

In [11]:
### Title
title = bookcard.find('span', {'class':"text-truncate"})

In [12]:
title.text

'magajin wilbafos [chapter 272] : zaren laidán'

In [13]:
###Price
price= bookcard.find("strong")

In [14]:
price.text

'₦10.00'

In [15]:
price.text[1:-3]

'10'

In [16]:
bookcard.find('strong').text[1:-3]

'10'

In [17]:
### Author's name

Author= bookcard.find('h5', {'class':"name"})

In [18]:
Author.text

'by A M Ibrahim '

In [19]:
Author.text[3:]

'A M Ibrahim '

In [20]:
### No of reads
read= bookcard.find('div', {'class':"reads"})

In [21]:
read.text

'22 reads'

In [22]:
read.text[:-5]

'22 '

In [23]:
### Rating's score
rating = bookcard.find('div', {'title':"ratings"})

In [24]:
rating.text

'0'

In [25]:
### Blurb/ Book description
blurb = bookcard.find('p', {'class':"description"})

In [26]:
blurb.text

'Labarin Izza!! ...'

In [27]:
### The link to the book is embedded in the 'a' tag while book cover is in the 'img' tage
bookcard.find_all('a')

[<a href="/book/about/magajin_wilbafos_chapter_272_zaren_laid_n/30045"><img alt="Magajin Wilbafos [Chapter 272] : Zaren Laidán" src="https://okada-assets-production.s3.eu-west-2.amazonaws.com/applications/content/images/bookImages/73e5b10c1a1b5396348bcd6b96bfc95d.jpg"/></a>,
 <a href="/book/about/magajin_wilbafos_chapter_272_zaren_laid_n/30045"><span class="text-truncate">magajin wilbafos [chapter 272] : zaren laidán</span></a>,
 <a href="https://okadabooks.com/user/Magajinwilbafos"><div class="avatar"><img alt="Author Image" src="https://okada-assets-production.s3.eu-west-2.amazonaws.com/applications/content/images/profilePhotos/c050c5b05c47955d3e7cff75aba5776d12f6e507.jpg"/></div><h5 class="name">by A M Ibrahim </h5></a>,
 <a href="/book/about/magajin_wilbafos_chapter_272_zaren_laid_n/30045"><span class=""><span>Available to read on app and web</span><i class="icon fa fa-dribbble"></i></span></a>]

In [28]:
###About the book(Link to a summary of the book)
bookcard.find('a')['href']

'/book/about/magajin_wilbafos_chapter_272_zaren_laid_n/30045'

In [29]:
base = 'https://okadabooks.com'

In [30]:
base + bookcard.find('a')['href'] 

'https://okadabooks.com/book/about/magajin_wilbafos_chapter_272_zaren_laid_n/30045'

---

## Part 2 : Automation

In each category, there is a load more button which when clicked on, reveals more bookcards. I am going to use selenium to click on the button till it reaches the end of the page and it is no longer available. This is when dynamic scraping begins.

In [1]:
from selenium import webdriver

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito') 

## In the final forloop, a headless argument will be used in order not to open the browser when 
## automation is going on. Incognito option means an incognito window will be used.

In [10]:
### The executable path is where I my chrome driver is located in my browser. Change it to yours.

driver=webdriver.Chrome(executable_path= 'C:/Users/ru/Downloads/Programs/chromedriver', chrome_options=options)

C:\Users\ru\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
driver.get('https://okadabooks.com/category/fiction/5')

In [6]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

In [12]:
### Get the css path by right clicking on the html tag gotten from inspecting the load more button.
### This clicks it once. 

okada= driver.find_element_by_css_selector('#_app > div.main-wrapper > main > div > main > div > div.col-lg-9 > div > div.container.d-flex.justify-content-center.mb-5 > button')
okada.click()

In [8]:
### This is to continue pressing the load more button till it is no longer there. It will be commented
### here. Check final forloop for usage. Action chains help in scrolling to reveal the element (which
## is the load button in this case.). time.sleep() is an extreme case of webdriver wait which is used
## here. Selenium exceptions helps to cater for instances of errors. 

import time 

while True:
    try:
        element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '#_app > div.main-wrapper > main > div > main > div > div.col-lg-9 > div > div.container.d-flex.justify-content-center.mb-5 > button'))
        WebDriverWait(driver, 10).until(element_present)
        load_button= driver.find_element_by_css_selector('#_app > div.main-wrapper > main > div > main > div > div.col-lg-9 > div > div.container.d-flex.justify-content-center.mb-5 > button')
        load_button.click()
    except ElementClickInterceptedException:     
        ActionChains(driver).move_to_element(load_button).click().perform()
    except TimeoutException:
        print("Timeout exception")
        break

Timeout exception


## Part 3: Getting all the urls for each category

Instead of typing all the urls for each category, it can easily gottten from the html tag. On okadbooks.com/store, right-click and inspect the category wrapper (where you click, it redirects you to the page of the category.)


In [3]:
import pandas as pd

In [3]:
import requests
from bs4 import BeautifulSoup

In [6]:
r = requests.get("https://okadabooks.com/store", headers=headers)
soup= BeautifulSoup(r.text, 'html.parser')

In [7]:
soup.find_all('a')[8]['href']

'/category/science_fiction/1'

In [39]:
### Corresponding category can be gotten from the url
soup.find_all('a')[8]['href'][10:-2]

'science_fiction'

In [8]:
base = 'https://okadabooks.com'
urls=[]
category= []
for i in range(8,30):
    urls.append(base + soup.find_all('a')[i]['href'])
    if i <=16:
        category.append(soup.find_all('a')[i]['href'][10:-2])
    else:
        category.append(soup.find_all('a')[i]['href'][10:-3])

In [9]:
urls

['https://okadabooks.com/category/science_fiction/1',
 'https://okadabooks.com/category/business_finance/2',
 'https://okadabooks.com/category/religion_and_beliefs/3',
 'https://okadabooks.com/category/self_help/4',
 'https://okadabooks.com/category/mystery/5',
 'https://okadabooks.com/category/educational/6',
 'https://okadabooks.com/category/memoir/7',
 'https://okadabooks.com/category/fiction/8',
 'https://okadabooks.com/category/non-fiction/9',
 'https://okadabooks.com/category/health_living/10',
 'https://okadabooks.com/category/other/11',
 'https://okadabooks.com/category/children/12',
 'https://okadabooks.com/category/romance/13',
 'https://okadabooks.com/category/adventure/14',
 'https://okadabooks.com/category/free/15',
 'https://okadabooks.com/category/raw_unpublished_works/16',
 'https://okadabooks.com/category/comics/17',
 'https://okadabooks.com/category/history/18',
 'https://okadabooks.com/category/drama/19',
 'https://okadabooks.com/category/poetry/20',
 'https://okadab

In [42]:
### Creating a dictionary with category as its value
cat= dict(zip(urls,category))

In [43]:
cat

{'https://okadabooks.com/category/science_fiction/1': 'science_fiction',
 'https://okadabooks.com/category/business_finance/2': 'business_finance',
 'https://okadabooks.com/category/religion_and_beliefs/3': 'religion_and_beliefs',
 'https://okadabooks.com/category/self_help/4': 'self_help',
 'https://okadabooks.com/category/mystery/5': 'mystery',
 'https://okadabooks.com/category/educational/6': 'educational',
 'https://okadabooks.com/category/memoir/7': 'memoir',
 'https://okadabooks.com/category/fiction/8': 'fiction',
 'https://okadabooks.com/category/non-fiction/9': 'non-fiction',
 'https://okadabooks.com/category/health_living/10': 'health_living',
 'https://okadabooks.com/category/other/11': 'other',
 'https://okadabooks.com/category/children/12': 'children',
 'https://okadabooks.com/category/romance/13': 'romance',
 'https://okadabooks.com/category/adventure/14': 'adventure',
 'https://okadabooks.com/category/free/15': 'free',
 'https://okadabooks.com/category/raw_unpublished_wor

---

## Part 4: Final Scraping

All sections of the code as explained above have been merged into a forloop that iterates through the 22 pages. If-else statements have been included in the extraction of bookcard details to cater for exceptions. example: Replacing 'K' with '000' for reads.

In [14]:
## important

import requests
from bs4 import BeautifulSoup
import time

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
r = requests.get("https://okadabooks.com/store", headers=headers)
soup= BeautifulSoup(r.text, 'html.parser')

base = 'https://okadabooks.com'
urls=[]
category= []

for i in range(8,30):
    urls.append(base + soup.find_all('a')[i]['href'])
    if i <=16:
        category.append(soup.find_all('a')[i]['href'][10:-2])
    else:
        category.append(soup.find_all('a')[i]['href'][10:-3])
        
cat= dict(zip(urls,category))  ##Dictionary of category and urls

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')  ## chrome will run in the background to make the process faster.
options.add_argument("user-agent='Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'")

prefs={"profile.managed_default_content_settings.images": 2} ##blocks all images on webpages, disk cache
options.add_experimental_option("prefs", prefs)
driver=webdriver.Chrome(executable_path= 'C:/Users/ru/Downloads/Programs/chromedriver', chrome_options=options)

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

Title=[]
Author=[]
Genre=[]
Price=[]
Ratings=[]
Reads=[]
Blurb= []
Booklink=[]



for url in urls:
    
    driver.get(url)
    
    driver.implicitly_wait(10)
    
    while True:
        try:
            element_present = EC.presence_of_element_located((By.CSS_SELECTOR, '#_app > div.main-wrapper > main > div > main > div > div.col-lg-9 > div > div.container.d-flex.justify-content-center.mb-5 > button'))
            WebDriverWait(driver, 10, ignored_exceptions=ignored_exceptions).until(element_present)
            load_button= driver.find_element_by_css_selector('#_app > div.main-wrapper > main > div > main > div > div.col-lg-9 > div > div.container.d-flex.justify-content-center.mb-5 > button')
            load_button.click()
        except ElementClickInterceptedException:     
            ActionChains(driver).move_to_element(load_button).click().perform()
        except StaleElementReferenceException:
            break
        except NoSuchElementException:
            break
        except TimeoutException:
            break
    
    time.sleep(5)
    
    
    page_source= driver.page_source ##get page source after clicking load more button it dissapers
    
    soup= BeautifulSoup(page_source, 'html.parser')
    
    bookcards= soup.find_all('div', {'class':'book__card'})
    bookcard= bookcards[0]

    
    for bookcard in bookcards:
        
        ##Title
        Title.append(bookcard.find('span', {'class':"text-truncate"}).text)
        
        ##Author
        Author.append(bookcard.find('h5', {'class':"name"}).text[3:])
        
        ##Genre
        Genre.append(cat[url])
        
        ##Price
        price= bookcard.find("strong").text
        if price=='Free':
            Price.append(0)
        else:
            Price.append(int(price[1:-3]))
        
        ### Ratings   
        Ratings.append(float(bookcard.find('div', {'title':"ratings"}).text))
        
        ##Reads
        Reads.append(int(bookcard.find('div', {'class':"reads"}).text[:-5].strip().replace('K','000')))
        
        
        ###Book Link
        Booklink.append(base + bookcard.find('a')['href'])
        
        
    time.sleep(1)
    
driver.quit()
    
print("i have succesfully scraped {} bookcards".format(len(Title)))

i have succesfully scraped 39562 bookcards


### Creating a dataframe 

In [1]:
import pandas as pd


In [16]:
cols=['Title','Author','Genre','Price','Reads', 'Ratings', 'Booklink']

In [17]:
df= pd.DataFrame({'Title':Title,'Author': Author,'Genre':Genre,'Price':Price, 'Reads':Reads, \
                  'Ratings':Ratings , 'Booklink': Booklink}, columns=cols)

In [80]:
df.head()

,Title,Author,Genre,Price,Reads,Ratings,Booklink
0,the long walk,Milton J Davis,science_fiction,360,0,0.0,https://okadabooks.com/book/about/the_long_walk/29737
1,bilisi: rebirth of the orisha #1.,Tomi Adegbite.,science_fiction,1500,0,0.0,https://okadabooks.com/book/about/bilisi_rebirth_of_the_orisha_1/29663
2,woman of the woods,Milton J Davis,science_fiction,1440,0,0.0,https://okadabooks.com/book/about/woman_of_the_woods/29664
3,badass,Razzy writer,science_fiction,0,11,0.0,https://okadabooks.com/book/about/badass/29642
4,priestess of nku,Milton J Davis,science_fiction,1440,0,0.0,https://okadabooks.com/book/about/priestess_of_nku/29626


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39562 entries, 0 to 18988
Data columns (total 7 columns):
Title       39562 non-null object
Author      39562 non-null object
Genre       39562 non-null object
Price       39562 non-null int64
Reads       39562 non-null int64
Ratings     39562 non-null float64
Booklink    39562 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 2.4+ MB


---

### Eliminating Duplicates

The data file to be exported should not have duplicates. The algorithm behind load more button suggests there will be duplicates. In the data analysis to be conducted, these duplicates will make my findings inaccurate. An easy way is to eliminate duplicates using the booklink column. This is preferred over the 'Title' column because there are books with similar names that are entirely different.

In [23]:
df_okadabooks = df.drop_duplicates(['Booklink'])

In [84]:
len(df_okadabooks)

14843

In [82]:
df_okadabooks.head()

,Title,Author,Genre,Price,Reads,Ratings,Booklink
0,the long walk,Milton J Davis,science_fiction,360,0,0.0,https://okadabooks.com/book/about/the_long_walk/29737
1,bilisi: rebirth of the orisha #1.,Tomi Adegbite.,science_fiction,1500,0,0.0,https://okadabooks.com/book/about/bilisi_rebirth_of_the_orisha_1/29663
2,woman of the woods,Milton J Davis,science_fiction,1440,0,0.0,https://okadabooks.com/book/about/woman_of_the_woods/29664
3,badass,Razzy writer,science_fiction,0,11,0.0,https://okadabooks.com/book/about/badass/29642
4,priestess of nku,Milton J Davis,science_fiction,1440,0,0.0,https://okadabooks.com/book/about/priestess_of_nku/29626


### Getting blurb

Here is a more efficient way to get the blurb. I went to each unique booklink and scraped the blurb. Let use the first booklink from the table above-- https://okadabooks.com/book/about/the_long_walk/29737 .

In [24]:
response= requests.get('https://okadabooks.com/book/about/badass/29642', headers=headers)

In [27]:
response

<Response [200]>

In [25]:
root= BeautifulSoup(response.text, 'html.parser')

In [9]:
root.find('div', {'class': 'book-main__content'})

<div class="book-main__content"><p>Patience de Verteuil left her home in Trinidad with her father for what she thought would be a short journey to America. Instead she finds herself embroiled in a struggle between powerful supernatural forces, in which an object in her possession means the difference between defeat or victory. The Long Walk is a Steamfunk tale that will stay with you long after the final words are read.

Noticed an error in this book? Send a mail to flag@okadabooks.com to report it</p></div>

In [10]:
root.find('div', {'class': 'book-main__content'}).text[:-80]

'Patience de Verteuil left her home in Trinidad with her father for what she thought would be a short journey to America. Instead she finds herself embroiled in a struggle between powerful supernatural forces, in which an object in her possession means the difference between defeat or victory. The Long Walk is a Steamfunk tale that will stay with you long after the final words are read.'

In [5]:
import requests
import time
from bs4 import BeautifulSoup

In [6]:
Booklink_list= df_okadabooks.Booklink.tolist()

In [7]:
def get_blurb():
    response= requests.get(booklink, headers=headers, timeout=600)
    root= BeautifulSoup(response.text, 'html.parser')
    blurb_content= root.find('div', {'class': 'book-main__content'}).text[:-80]
    return blurb_content

In [35]:
Blurb=[]
for booklink in Booklink_list:
    try:
        time.sleep(1)
        Blurb.append(get_blurb())
    except AttributeError:
        ### This is for if one of the urls is made unavailable because a book is removed 
        Blurb.append('Book not available')
    except requests.exceptions.ConnectionError as errc:
        time.sleep(4)
        Blurb.append(get_blurb())
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",booklink)
        time.sleep(3)
        Blurb.append(get_blurb())
    except AttributeError:
        ### This is for if one of the urls is made unavailable because a book is removed 
        Blurb.append('Book not available')
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err, booklink)
        Blurb.append('unknown error')   

print('\nsuccesfully gotten all blurbs')

Timeout Error: https://okadabooks.com/book/about/the_faith_of_a_prostitute/26735 
Timeout Error: https://okadabooks.com/book/about/bravery/28125 
Timeout Error: https://okadabooks.com/book/about/yar_gidan_tsohuwa/27634 
Timeout Error: https://okadabooks.com/book/about/the_chronicles_of_ola/15021 
Timeout Error: https://okadabooks.com/book/about/the_whence_and_the_whither_of_man/4660

succesfully gotten all blurbs


In [46]:
len(Blurb)

14843

In [17]:
cols= ['booklink' ,"blurb"]
blurb_df= pd.DataFrame({ 'Booklink': booklink, 'Blurb':Blurb }, columns=cols)

In [ ]:
blurb_df= blurb_df.drop(columns= 'Unnamed: 0') ##remove unnamed column

In [59]:
blurb_df.head(2)

,Booklink,Blurb
0,https://okadabooks.com/book/about/the_long_wal...,Patience de Verteuil left her home in Trinidad...
1,https://okadabooks.com/book/about/bilisi_rebir...,Enter a world or two of pure fantasy and wonde...


In [60]:
df_okadabooks.head(2)

,Title,Author,Genre,Price,Reads,Ratings,Booklink
0,the long walk,Milton J Davis,science_fiction,360,0,0.0,https://okadabooks.com/book/about/the_long_wal...
1,bilisi: rebirth of the orisha #1.,Tomi Adegbite.,science_fiction,1500,0,0.0,https://okadabooks.com/book/about/bilisi_rebir...


In [63]:
### Merging both dataframes into one 
okadabooks_df= pd.merge(df_okadabooks, blurb_df, how= 'inner', on= 'Booklink')

In [70]:
### Drop all 'Book not available' in blurb as a result of attribute error when trying to get the blurb
okadabooks_df= okadabooks_df[okadabooks_df.Blurb!="Book not available"]

In [64]:
### Help see the dataframe contents better
pd.set_option('display.max_colwidth', -1)

In [73]:
okadabooks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14803 entries, 0 to 14842
Data columns (total 8 columns):
Title       14803 non-null object
Author      14803 non-null object
Genre       14803 non-null object
Price       14803 non-null int64
Reads       14803 non-null int64
Ratings     14803 non-null float64
Booklink    14803 non-null object
Blurb       14802 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 1.0+ MB


In [72]:
okadabooks_df.head(3)

,Title,Author,Genre,Price,Reads,Ratings,Booklink,Blurb
0,the long walk,Milton J Davis,science_fiction,360,0,0.0,https://okadabooks.com/book/about/the_long_walk/29737,"Patience de Verteuil left her home in Trinidad with her father for what she thought would be a short journey to America. Instead she finds herself embroiled in a struggle between powerful supernatural forces, in which an object in her possession means the difference between defeat or victory. The Long Walk is a Steamfunk tale that will stay with you long after the final words are read."
1,bilisi: rebirth of the orisha #1.,Tomi Adegbite.,science_fiction,1500,0,0.0,https://okadabooks.com/book/about/bilisi_rebirth_of_the_orisha_1/29663,"Enter a world or two of pure fantasy and wonder, as Bilisi Soneye discovers herself in more than one reality."
2,woman of the woods,Milton J Davis,science_fiction,1440,0,0.0,https://okadabooks.com/book/about/woman_of_the_woods/29664,"Sadatina, an Adamu girl on the brink of becoming a woman has lived a peaceful life with her family in Adamusola, the land beyond the Old Men Mountains. But tragic events change her life forever, revealing a hidden past that leads her into the midst of a war between her people and those that would see them destroyed, the Mosele. Armed with a spiritual weapon and her feline 'sisters,' Sadatina becomes a Shosa, a warrior trained to fight the terrible nyokas, demon-like creatures that aid the Mosele in their war against her people. Woman of the Woods by Milton Davis is an action filled, emotionally charged adventure that expands the scope of the world of Uhuru and introduces another unforgettable character to the fabled continent's heroic legends."


### Exporting to a csv file

In [77]:
### change  the directory to yours
okadabooks_df.to_csv(r"C:\Users\ru\Desktop\Programming\PYTHONN!\Okadabooks\okadabooks.csv")